In [33]:
%classpath add jar ../../konduit.jar

In [34]:
%%bash
konduit stop tensorflow-mnist

No konduit server exists with an id: 'tensorflow-mnist'.



In [35]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import io.micrometer.prometheus.PrometheusConfig;
import io.micrometer.prometheus.PrometheusMeterRegistry;
import io.micrometer.core.instrument.Counter;
import io.micrometer.core.instrument.binder.jvm.JvmMemoryMetrics;
import io.micrometer.core.instrument.binder.system.ProcessorMetrics;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

public class PrometheusEndpoint implements Endpoint {

    public static PrometheusMeterRegistry registry;
    public static List<Counter> classCounterIncrement = new ArrayList();
    public static List<String> labels = Arrays.asList("Number_0", "Number_1", "Number_2", "Number_3", "Number_4", "Number_5", "Number_6", "Number_7", "Number_8", "Number_9");
    
    static {
        registry = new PrometheusMeterRegistry(PrometheusConfig.DEFAULT);
        
        if (registry != null) {
            System.out.println("Using metrics registry " + registry.getClass().getName() + " for inference");
            new JvmMemoryMetrics().bindTo(registry);
            new ProcessorMetrics().bindTo(registry);
            
            Counter serverUpTimeCounter = registry.counter("server.up.time");
            double increment = 5.0;
            new Timer().schedule(new TimerTask() {
                @Override
                public void run() {
                    serverUpTimeCounter.increment(increment);
                }
            }, 5000, ((int) increment) * 1000);
            
            
            for (String label : labels) {
                classCounterIncrement.add(Counter.builder(label)
                        .description("Classification counts seen so far for class label: " + label)
                        .baseUnit("classification.outcome")
                        .register(registry));
            }
        } else {
            System.out.println("Not using metrics registry.");
        }
    }
    
    private PipelineExecutor pipelineExecutor;

    public PrometheusEndpoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/tensorflow-metrics"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("text/plain; version=0.0.4; charset=utf-8"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> handler.response().putHeader(HttpHeaders.CONTENT_TYPE, "text/plain; version=0.0.4; charset=utf-8").end(registry.scrape());
    }
}

ai.konduit.PrometheusEndpoint

In [38]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;

import io.micrometer.core.instrument.Counter;

import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    private Counter requestsHandled = PrometheusEndpoint.registry.counter("requests.handled");
    
    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                INDArray output = exec.getNDArray("output_layer/Softmax").getAs(INDArray.class);
                handler.response().end(ObjectMappers.toJson(output.data().asFloat()));
                taskHandler.complete();
                
                requestsHandled.increment();
                
                INDArray argMax = Nd4j.argMax(output, -1);
                for(int i = 0; i < argMax.length(); i++) {
                    PrometheusEndpoint.classCounterIncrement.get(argMax.getInt(i)).increment();
                }
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [39]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {
    
    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor), new PrometheusEndpoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [40]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("Saved %s at: %s%n", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker2879603904354261793/outDir
/root/konduit/konduit.jar
Saved /tmp/beaker2879603904354261793/outDir:/root/konduit/konduit.jar at: /root/konduit/demos/4-tensorflow-mnist/classpath


null

In [41]:
%%bash
java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id tensorflow-mnist -c tensorflow.json -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker2879603904354261793/outDir:/root/konduit/konduit.jar
INFO: Running command /opt/conda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/tensorflow-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_a5f0e8c62a894ea9.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c tensorflow.json -Dserving.id=tensorflow-mnist
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs tensorflow-mnist'



In [42]:
%%bash
konduit logs tensorflow-mnist -l 1000

06:24:55.284 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/4-tensorflow-mnist/tensorflow.json
06:24:55.291 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
06:24:55.293 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
06:24:55.293 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'tensorflow-mnist'
06:24:55.555 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 27 PipelineStepRunnerFactory instances
06:24:56.099 [vert.x-worker-thread-0] INFO  a.k.s.v.verticle.InferenceVerticle - 

####################################################################
#                                                                  #
#    |  /   _ \   \ |  _ \  |  | _ _| __ __|    |  /     |  /      #
#    . <   (   | .  |  |  | |  |   |     |      . <    

In [43]:
%%html
  <div id="banner">
    <div style="display: inline-block">
        <img src="test_files/test_input_number_0.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_1.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_2.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_3.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_4.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_5.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_6.png"/>
    </div>

    <div style="display: inline-block">
        <img src="test_files/test_input_number_7.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_8.png"/>
    </div>
      
    <div style="display: inline-block">
        <img src="test_files/test_input_number_9.png"/>
    </div>
      
</div>

In [44]:
%%bash
for i in {1..100}
do
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_0.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_1.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_2.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_3.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_4.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_5.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_6.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_7.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_8.png' http://localhost:9008/infer && echo ""
   echo -e $i: '\c' && curl -s -H 'Content-Type: multipart/form-data' -X POST -F 'image=@test_files/test_input_number_9.png' http://localhost:9008/infer && echo ""
done

1: [ 0.99999714, 1.3149393E-9, 3.6921335E-8, 8.5948994E-13, 7.267632E-10, 1.0761406E-6, 1.7526992E-6, 1.8965244E-12, 7.2676766E-13, 6.594134E-10 ]
1: [ 8.562342E-6, 0.9987562, 2.3001013E-8, 3.0561345E-10, 0.0012088065, 7.575954E-8, 4.2052684E-6, 4.51372E-6, 7.941958E-11, 1.7574817E-5 ]
1: [ 1.8688723E-8, 0.096255526, 0.77537984, 1.7737483E-8, 0.12277348, 4.749891E-13, 3.0434855E-6, 0.0055881594, 7.3292995E-12, 3.453348E-10 ]
1: [ 5.03358E-13, 1.2224174E-7, 1.530677E-8, 0.9999672, 7.666789E-14, 2.8814886E-8, 1.0756101E-17, 3.0170297E-5, 7.513803E-9, 2.4812875E-6 ]
1: [ 1.4776832E-12, 2.3744076E-7, 1.7927584E-7, 2.2138133E-17, 0.9998833, 1.2808428E-15, 1.0593954E-12, 1.16203046E-4, 4.836174E-15, 8.583544E-11 ]
1: [ 4.9618645E-8, 0.9965842, 1.1146856E-7, 4.5927404E-8, 0.003301259, 1.0254329E-6, 4.0990802E-8, 1.08784516E-4, 2.0992433E-8, 4.3916148E-6 ]
1: [ 1.0224228E-6, 2.248694E-6, 3.9290812E-6, 5.5416473E-14, 9.053977E-5, 2.6707519E-12, 0.99990225, 3.3465622E-10, 3.8954322E-13, 5.668778

## Metrics
Visit: http://localhost:3000/d/lP_JcnHWz/pipeline-metrics?orgId=1&refresh=5s to view metrics